In [26]:
import torch
import pytorch_lightning as pl
from torch import nn
from torchmetrics import Accuracy



class LinearNetwork(pl.LightningModule):
    def __init__(self, input_features, output_features, lr):
        super().__init__()
        barrel_size = input_features * 4
        self.lr = lr
        self.model = nn.Sequential(
            nn.Linear(input_features, barrel_size),
            nn.Linear(barrel_size, barrel_size),
            nn.Linear(barrel_size, output_features)
        )
        
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task='multiclass', num_classes=2)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
        
    def forward(self, X):
        return self.model(X)
    
    def training_step(self, batch, batch_idx) -> torch.Tensor:
        X, y = batch
        y_hat = self.model(X)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.model(X)
        loss = nn.functional.cross_entropy(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log("test loss", loss)
        self.log("test accuracy", accuracy)
        return {"test_loss":loss, "test_accuracy": accuracy}


class XORDataset(torch.utils.data.Dataset):
    def __init__(self, size=1000):
        self.size = size
        self.input = torch.Tensor([[0,0],[0,1],[1,0],[1,1]])
        self.labels = torch.Tensor([0, 1, 1, 0]).long()
        self.len = self.input.shape[0]

        self.xor_table = [(i, j, int(bool(i) != bool(j))) for i in range(2) for j in range(2)]

    def __getitem__(self, idx):
        idx = idx % self.len
        return (self.input[idx], self.labels[idx]) 

    def __len__(self):
        return self.size




class ANDDataset(torch.utils.data.Dataset):
    def __init__(self, size=1000):
        self.size = size
        self.input = torch.Tensor([[0,0],[0,1],[1,0],[1,1]])
        self.labels = torch.Tensor([0, 0, 0, 1]).long()
        self.len = self.input.shape[0]

        self.xor_table = [(i, j, int(bool(i) != bool(j))) for i in range(2) for j in range(2)]

    def __getitem__(self, idx):
        idx = idx % self.len
        return (self.input[idx], self.labels[idx]) 

    def __len__(self):
        return self.size




In [27]:
from torch.utils import data
xor_model = LinearNetwork(2,2,lr=3e-4)
and_model = LinearNetwork(2,2,lr=3e-4)

xor_dataset = XORDataset()
and_dataset = ANDDataset()

xor_train_loader = data.DataLoader(xor_dataset, batch_size=4)
and_train_loader = data.DataLoader(and_dataset, batch_size=4)

xor_trainer = pl.Trainer(max_epochs=100)
and_trainer = pl.Trainer(max_epochs=100)

xor_trainer.fit(xor_model, train_dataloaders=xor_train_loader, val_dataloaders=xor_train_loader)
and_trainer.fit(and_model, train_dataloaders=and_train_loader, val_dataloaders=and_train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/1111108/miniforge3/envs/pt_ml/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:106: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")

  | Name     | Type               | Params
------------------------------------------------
0 | model    | Sequential         | 114   
1 | loss     | CrossEntropyLoss   | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
114       Trainable params
0         Non-trainable params
114       Total params
0.000     Total estimated model 

Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 426.92it/s, loss=0.693, v_num=32]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 425.03it/s, loss=0.693, v_num=32]



  | Name     | Type               | Params
------------------------------------------------
0 | model    | Sequential         | 114   
1 | loss     | CrossEntropyLoss   | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
114       Trainable params
0         Non-trainable params
114       Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 442.02it/s, loss=0, v_num=33]       

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 439.88it/s, loss=0, v_num=33]


In [28]:
xor_trainer.test(xor_model, dataloaders=xor_train_loader)
and_trainer.test(and_model, dataloaders=and_train_loader)

/Users/1111108/miniforge3/envs/pt_ml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 250/250 [00:00<00:00, 709.64it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test accuracy                 0.5
        test loss           0.6931461095809937
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 250/250 [00:00<00:00, 757.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test accuracy                 1.0
        test loss                 

[{'test loss': 0.0, 'test accuracy': 1.0}]

- linear layer만으로 classification model을 만들었을때...
  - AND는 거의 완벽한 fit를 보여줌
  - XOR는 같은 조건에서 0.5의 정확도를 보여줌

- 똑같은 구조에 각각의 Linear Layer사이에 Non-Linear Layer를 추가해서 Model에 Non-linearity를 추가함

In [29]:


class NonLinearNetwork(pl.LightningModule):
    def __init__(self, input_features, output_features, lr):
        super().__init__()
        barrel_size = input_features * 4
        self.lr = lr
        self.model = nn.Sequential(
            nn.Linear(input_features, barrel_size),
            nn.ReLU(),
            nn.Linear(barrel_size, barrel_size),
            nn.ReLU(),
            nn.Linear(barrel_size, output_features)
        )
        
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task='multiclass', num_classes=2)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)
        
    def forward(self, X):
        return self.model(X)
    
    def training_step(self, batch, batch_idx) -> torch.Tensor:
        X, y = batch
        y_hat = self.model(X)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.model(X)
        loss = nn.functional.cross_entropy(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log("val_loss", loss)
        self.log("val_accuracy", accuracy)
        return {"val_loss":loss, "val_accuracy": accuracy}

    def test_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.model(X)
        loss = nn.functional.cross_entropy(y_hat, y)
        accuracy = self.accuracy(y_hat, y)
        self.log("test loss", loss)
        self.log("test accuracy", accuracy)
        return {"test_loss":loss, "test_accuracy": accuracy}

In [30]:
versatile_xor_fitter = NonLinearNetwork(2,2, lr=2e-4)
versatile_and_fitter = NonLinearNetwork(2,2, lr=2e-4)

xor_trainer = pl.Trainer(max_epochs=100)
and_trainer = pl.Trainer(max_epochs=100)

xor_trainer.fit(model=versatile_xor_fitter, train_dataloaders=xor_train_loader)
and_trainer.fit(model=versatile_and_fitter, train_dataloaders=and_train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/1111108/miniforge3/envs/pt_ml/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name     | Type               | Params
------------------------------------------------
0 | model    | Sequential         | 114   
1 | loss     | CrossEntropyLoss   | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
114       Trainable params
0         Non-trainable params
114       Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 425.52it/s, loss=0.000243, v_num=34]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 423.17it/s, loss=0.000243, v_num=34]



  | Name     | Type               | Params
------------------------------------------------
0 | model    | Sequential         | 114   
1 | loss     | CrossEntropyLoss   | 0     
2 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
114       Trainable params
0         Non-trainable params
114       Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 434.89it/s, loss=0.00034, v_num=35] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 250/250 [00:00<00:00, 431.94it/s, loss=0.00034, v_num=35]


In [31]:
and_trainer.test(model=versatile_and_fitter, dataloaders=and_train_loader)
xor_trainer.test(model=versatile_xor_fitter, dataloaders=xor_train_loader)

Testing DataLoader 0: 100%|██████████| 250/250 [00:00<00:00, 733.89it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test accuracy                 1.0
        test loss          0.0003390689962543547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 250/250 [00:00<00:00, 746.26it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test accuracy                 1.0
        test loss         0.0002

[{'test loss': 0.00024202543136198074, 'test accuracy': 1.0}]